## Competição DSA 2019/06
<h3>Prever Índice de Lealdade</h3>

In [1]:
# Importar as bibliotecas necessárias para este notebook
import pandas as pd
import numpy  as np
from datetime import datetime as dt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import mean_squared_error
from sklearn.decomposition   import PCA
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LinearRegression
import warnings
warnings.filterwarnings("ignore")

In [2]:
# importar os arquivos para dataframes:
train_file        = pd.read_csv('dataset_treino.csv')
test_file         = pd.read_csv('dataset_teste.csv')
comerciantes_file = pd.read_csv('comerciantes.csv')
historico_cartoes = pd.read_csv('transacoes_historicas.csv')
historico_comerci = pd.read_csv('novas_transacoes_comerciantes.csv')

In [3]:
# Visualizar as primeiras linhas dos arquivos:
train_file.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056


In [4]:
test_file.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1


In [5]:
comerciantes_file.head(3)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0


In [6]:
historico_cartoes.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
1,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
2,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34


In [7]:
historico_comerci.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14


In [8]:
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
first_active_month    201917 non-null object
card_id               201917 non-null object
feature_1             201917 non-null int64
feature_2             201917 non-null int64
feature_3             201917 non-null int64
target                201917 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ MB


In [9]:
train_file['card_id'].describe()

count              201917
unique             201917
top       C_ID_99d6e41170
freq                    1
Name: card_id, dtype: object

In [10]:
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123623 entries, 0 to 123622
Data columns (total 5 columns):
first_active_month    123622 non-null object
card_id               123623 non-null object
feature_1             123623 non-null int64
feature_2             123623 non-null int64
feature_3             123623 non-null int64
dtypes: int64(3), object(2)
memory usage: 4.7+ MB


In [11]:
test_file.card_id.describe()

count              123623
unique             123623
top       C_ID_1850c3004f
freq                    1
Name: card_id, dtype: object

In [12]:
comerciantes_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 22 columns):
merchant_id                    334696 non-null object
merchant_group_id              334696 non-null int64
merchant_category_id           334696 non-null int64
subsector_id                   334696 non-null int64
numerical_1                    334696 non-null float64
numerical_2                    334696 non-null float64
category_1                     334696 non-null object
most_recent_sales_range        334696 non-null object
most_recent_purchases_range    334696 non-null object
avg_sales_lag3                 334683 non-null float64
avg_purchases_lag3             334696 non-null float64
active_months_lag3             334696 non-null int64
avg_sales_lag6                 334683 non-null float64
avg_purchases_lag6             334696 non-null float64
active_months_lag6             334696 non-null int64
avg_sales_lag12                334683 non-null float64
avg_purchases_lag12    

In [13]:
# comerciantes possui id repetido
comerciantes_file['merchant_id'].describe()

count              334696
unique             334633
top       M_ID_30340088f2
freq                    4
Name: merchant_id, dtype: object

In [14]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_c2b9ac2ea4']

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
210653,M_ID_c2b9ac2ea4,69907,554,25,-0.057471,-0.057471,Y,D,E,1.11,...,1.11,0.702381,3,1.11,0.702381,3,Y,-1,-1,NaN
210654,M_ID_c2b9ac2ea4,35,554,25,-0.057471,-0.057471,Y,D,E,1.11,...,1.11,0.702381,3,1.11,0.702381,3,Y,-1,-1,NaN
210655,M_ID_c2b9ac2ea4,35,554,25,-0.057471,-0.057471,Y,C,D,2.13,...,2.02,1.572673,6,1.77,1.491291,12,Y,-1,-1,NaN
210656,M_ID_c2b9ac2ea4,69907,554,25,-0.057471,-0.057471,Y,C,D,2.13,...,2.02,1.572673,6,1.77,1.491291,12,Y,-1,-1,NaN


In [15]:
comerciantes_file['most_recent_sales_range'].value_counts()

E    177104
D    117475
C     34075
B      5037
A      1005
Name: most_recent_sales_range, dtype: int64

In [16]:
historico_cartoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 14 columns):
authorized_flag         199999 non-null object
card_id                 199999 non-null object
city_id                 199999 non-null int64
category_1              199999 non-null object
installments            199999 non-null int64
category_3              199390 non-null object
merchant_category_id    199999 non-null int64
merchant_id             199075 non-null object
month_lag               199999 non-null int64
purchase_amount         199999 non-null float64
purchase_date           199999 non-null object
category_2              185232 non-null float64
state_id                199999 non-null int64
subsector_id            199999 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 21.4+ MB


In [17]:
historico_comerci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1963031 entries, 0 to 1963030
Data columns (total 14 columns):
authorized_flag         object
card_id                 object
city_id                 int64
category_1              object
installments            int64
category_3              object
merchant_category_id    int64
merchant_id             object
month_lag               int64
purchase_amount         float64
purchase_date           object
category_2              float64
state_id                int64
subsector_id            int64
dtypes: float64(2), int64(6), object(6)
memory usage: 209.7+ MB


In [18]:
# Visualizar os dados estatísticos dos arquivos:
train_file.describe()

,feature_1,feature_2,feature_3,target
count,201917.000000,201917.000000,201917.000000,201917.000000
mean,3.105311,1.745410,0.565569,-0.393636
std,1.186160,0.751362,0.495683,3.850500
min,1.000000,1.000000,0.000000,-33.219281
25%,2.000000,1.000000,0.000000,-0.883110
50%,3.000000,2.000000,1.000000,-0.023437
75%,4.000000,2.000000,1.000000,0.765453
max,5.000000,3.000000,1.000000,17.965068


In [19]:
test_file.describe()

,feature_1,feature_2,feature_3
count,123623.00000,123623.000000,123623.000000
mean,3.10926,1.741796,0.564377
std,1.18911,0.749195,0.495840
min,1.00000,1.000000,0.000000
25%,2.00000,1.000000,0.000000
50%,3.00000,2.000000,1.000000
75%,4.00000,2.000000,1.000000
max,5.00000,3.000000,1.000000


In [20]:
comerciantes_file.describe()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,city_id,state_id,category_2
count,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334683.000000,3.346960e+05,334696.000000,3.346830e+05,3.346960e+05,334696.000000,3.346830e+05,3.346960e+05,334696.000000,334696.000000,334696.000000,322809.000000
mean,31028.736143,423.131663,25.116404,0.011476,0.008103,13.832993,inf,2.994108,2.165079e+01,inf,5.947397,2.522771e+01,inf,11.599335,102.917926,11.860942,2.380002
std,31623.043426,252.898046,9.807371,1.098154,1.070497,2395.489999,NaN,0.095247,3.947108e+03,NaN,0.394936,5.251842e+03,NaN,1.520138,107.090673,6.176889,1.562661
min,1.000000,-1.000000,-1.000000,-0.057471,-0.057471,-82.130000,3.334953e-01,1.000000,-8.213000e+01,1.670447e-01,1.000000,-8.213000e+01,9.832954e-02,1.000000,-1.000000,-1.000000,1.000000
25%,3612.000000,222.000000,19.000000,-0.057471,-0.057471,0.880000,9.236499e-01,3.000000,8.500000e-01,9.022475e-01,6.000000,8.500000e-01,8.983333e-01,12.000000,-1.000000,9.000000,1.000000
50%,19900.000000,373.000000,27.000000,-0.057471,-0.057471,1.000000,1.016667e+00,3.000000,1.010000e+00,1.026961e+00,6.000000,1.020000e+00,1.043361e+00,12.000000,69.000000,9.000000,2.000000
75%,51707.250000,683.000000,33.000000,-0.047556,-0.047556,1.160000,1.146522e+00,3.000000,1.230000e+00,1.215575e+00,6.000000,1.290000e+00,1.266480e+00,12.000000,182.000000,16.000000,4.000000
max,112586.000000,891.000000,41.000000,183.735111,182.079322,851844.640000,inf,3.000000,1.513959e+06,inf,6.000000,2.567408e+06,inf,12.000000,347.000000,24.000000,5.000000


In [21]:
historico_cartoes.describe()

,city_id,installments,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id
count,199999.000000,199999.000000,199999.000000,199999.000000,199999.000000,185232.000000,199999.000000,199999.000000
mean,125.771759,0.495452,468.656738,-5.089705,-0.470251,2.150006,10.781014,27.631368
std,100.633496,0.927371,248.674052,3.712166,32.755567,1.518472,6.168744,9.492934
min,-1.000000,-1.000000,-1.000000,-13.000000,-0.746893,1.000000,-1.000000,-1.000000
25%,53.000000,0.000000,278.000000,-8.000000,-0.722445,1.000000,9.000000,19.000000
50%,88.000000,0.000000,437.000000,-5.000000,-0.694315,1.000000,9.000000,33.000000
75%,200.000000,1.000000,705.000000,-2.000000,-0.622488,3.000000,16.000000,34.000000
max,347.000000,12.000000,891.000000,0.000000,11269.666518,5.000000,24.000000,41.000000


In [22]:
historico_comerci.describe()

,city_id,installments,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id
count,1.963031e+06,1.963031e+06,1.963031e+06,1.963031e+06,1.963031e+06,1.851286e+06,1.963031e+06,1.963031e+06
mean,1.343867e+02,6.829643e-01,4.309701e+02,1.476515e+00,-5.509690e-01,2.197841e+00,1.088067e+01,2.597624e+01
std,1.015152e+02,1.584069e+00,2.463385e+02,4.994483e-01,6.940043e-01,1.528125e+00,6.038542e+00,1.012908e+01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,1.000000e+00,-7.468928e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,6.900000e+01,0.000000e+00,2.780000e+02,1.000000e+00,-7.166294e-01,1.000000e+00,9.000000e+00,1.900000e+01
50%,1.100000e+02,1.000000e+00,3.670000e+02,1.000000e+00,-6.748406e-01,1.000000e+00,9.000000e+00,2.900000e+01
75%,2.120000e+02,1.000000e+00,6.830000e+02,2.000000e+00,-5.816162e-01,3.000000e+00,1.500000e+01,3.400000e+01
max,3.470000e+02,9.990000e+02,8.910000e+02,2.000000e+00,2.631575e+02,5.000000e+00,2.400000e+01,4.100000e+01


In [23]:
train_file.isna().sum()

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64

In [24]:
test_file.isna().sum()

first_active_month    1
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64

In [25]:
test_file.first_active_month.describe()

count      123622
unique         75
top       2017-09
freq         8805
Name: first_active_month, dtype: object

In [26]:
# Criar variáveis dummies para as categorias:
train_file = pd.get_dummies(train_file, columns = ['feature_1', 'feature_2', 'feature_3'])
test_file = pd.get_dummies(test_file, columns = ['feature_1', 'feature_2', 'feature_3']); test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123623 entries, 0 to 123622
Data columns (total 12 columns):
first_active_month    123622 non-null object
card_id               123623 non-null object
feature_1_1           123623 non-null uint8
feature_1_2           123623 non-null uint8
feature_1_3           123623 non-null uint8
feature_1_4           123623 non-null uint8
feature_1_5           123623 non-null uint8
feature_2_1           123623 non-null uint8
feature_2_2           123623 non-null uint8
feature_2_3           123623 non-null uint8
feature_3_0           123623 non-null uint8
feature_3_1           123623 non-null uint8
dtypes: object(2), uint8(10)
memory usage: 3.1+ MB


In [27]:
# Criar novas colunas no tipo datetime
test_file['date_Card_Activation'] = pd.to_datetime(test_file['first_active_month'], format = "%Y-%m")
train_file['date_Card_Activation'] = pd.to_datetime(train_file['first_active_month'], format = "%Y-%m")

In [28]:
# remover colunas desnecessárias
train_file.drop(['first_active_month'], axis=1, inplace = True)
test_file.drop(['first_active_month'], axis=1, inplace = True)

In [29]:
# substituir valores NA do campo data pelo mais recente
test_file['date_Card_Activation'].fillna(test_file.date_Card_Activation.max(), inplace = True); test_file.isna().sum()

card_id                 0
feature_1_1             0
feature_1_2             0
feature_1_3             0
feature_1_4             0
feature_1_5             0
feature_2_1             0
feature_2_2             0
feature_2_3             0
feature_3_0             0
feature_3_1             0
date_Card_Activation    0
dtype: int64

In [30]:
comerciantes_file.isna().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [31]:
# Comparar se conteúdo das listas são iguais
avg_sales_lag3  = list(comerciantes_file[comerciantes_file['avg_sales_lag3'].isna()]['merchant_id'])
avg_sales_lag6  = list(comerciantes_file[comerciantes_file['avg_sales_lag6'].isna()]['merchant_id'])
avg_sales_lag12 = list(comerciantes_file[comerciantes_file['avg_sales_lag12'].isna()]['merchant_id'])
lista_final1 = list(set(avg_sales_lag3) - set(avg_sales_lag6)); lista_final1

[]

In [32]:
# Comparar se conteúdo das listas são iguais
lista_final2 = list(set(avg_sales_lag3) - set(avg_sales_lag12)); lista_final2

[]

In [33]:
# Substituir valores NA por zero
comerciantes_file['avg_sales_lag3'].fillna(0, inplace = True)
comerciantes_file['avg_sales_lag6'].fillna(0, inplace = True)
comerciantes_file['avg_sales_lag12'].fillna(0, inplace = True);
comerciantes_file.isna().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                     0
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                     0
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                    0
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [34]:
# Tratar valores NA em category_2
comerciantes_file['category_2'].fillna(0, inplace = True); comerciantes_file.isna().sum()

merchant_id                    0
merchant_group_id              0
merchant_category_id           0
subsector_id                   0
numerical_1                    0
numerical_2                    0
category_1                     0
most_recent_sales_range        0
most_recent_purchases_range    0
avg_sales_lag3                 0
avg_purchases_lag3             0
active_months_lag3             0
avg_sales_lag6                 0
avg_purchases_lag6             0
active_months_lag6             0
avg_sales_lag12                0
avg_purchases_lag12            0
active_months_lag12            0
category_4                     0
city_id                        0
state_id                       0
category_2                     0
dtype: int64

In [35]:
comerciantes_file.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,0.00,...,0.00,4.666667,6,0.00,3.833333,12,Y,-1,-1,0.0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,0.00,...,0.00,0.361111,6,0.00,0.347222,12,Y,-1,-1,0.0


In [36]:
# identificar range de valores das colunas referente a meses em atividade
print(comerciantes_file.active_months_lag3.unique())
print(comerciantes_file.active_months_lag6.unique())
print(comerciantes_file.active_months_lag12.unique())

[3 2 1]
[6 2 1 3 4 5]
[12  2 11  7  1  3 10  9  4  6  8  5]


In [37]:
# armazenar a razão de meses em atividade
comerciantes_file['qtty_active_months_lag3'] = comerciantes_file.active_months_lag3 / 3
comerciantes_file['qtty_active_months_lag6'] = comerciantes_file.active_months_lag6 / 6
comerciantes_file['qtty_active_months_lag12'] = comerciantes_file.active_months_lag12 / 12

In [38]:
# excluir colunas já tratadas acima
comerciantes_file.drop(['active_months_lag3', 'active_months_lag6', 'active_months_lag12'], axis=1, inplace = True); comerciantes_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 22 columns):
merchant_id                    334696 non-null object
merchant_group_id              334696 non-null int64
merchant_category_id           334696 non-null int64
subsector_id                   334696 non-null int64
numerical_1                    334696 non-null float64
numerical_2                    334696 non-null float64
category_1                     334696 non-null object
most_recent_sales_range        334696 non-null object
most_recent_purchases_range    334696 non-null object
avg_sales_lag3                 334696 non-null float64
avg_purchases_lag3             334696 non-null float64
avg_sales_lag6                 334696 non-null float64
avg_purchases_lag6             334696 non-null float64
avg_sales_lag12                334696 non-null float64
avg_purchases_lag12            334696 non-null float64
category_4                     334696 non-null object
city_id             

In [39]:
# As colunas cujo conceito se refere a setor de atuação, grupo econômico e região serão excluídas para evitar redundância
comerciantes_file.drop(['merchant_group_id', 'merchant_category_id', 'subsector_id', 'city_id', 'state_id'], axis=1, inplace = True); comerciantes_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 17 columns):
merchant_id                    334696 non-null object
numerical_1                    334696 non-null float64
numerical_2                    334696 non-null float64
category_1                     334696 non-null object
most_recent_sales_range        334696 non-null object
most_recent_purchases_range    334696 non-null object
avg_sales_lag3                 334696 non-null float64
avg_purchases_lag3             334696 non-null float64
avg_sales_lag6                 334696 non-null float64
avg_purchases_lag6             334696 non-null float64
avg_sales_lag12                334696 non-null float64
avg_purchases_lag12            334696 non-null float64
category_4                     334696 non-null object
category_2                     334696 non-null float64
qtty_active_months_lag3        334696 non-null float64
qtty_active_months_lag6        334696 non-null float64
qtty_active_mo

In [40]:
# Analisando outros campos quantitativos
print(comerciantes_file.numerical_1.describe())
print(comerciantes_file.numerical_2.describe())

count    334696.000000
mean          0.011476
std           1.098154
min          -0.057471
25%          -0.057471
50%          -0.057471
75%          -0.047556
max         183.735111
Name: numerical_1, dtype: float64
count    334696.000000
mean          0.008103
std           1.070497
min          -0.057471
25%          -0.057471
50%          -0.057471
75%          -0.047556
max         182.079322
Name: numerical_2, dtype: float64


In [41]:
# Analisando outros campos qualitativos
print(comerciantes_file.category_1.describe())
print(comerciantes_file.category_4.describe())
print(comerciantes_file.category_2.describe())
print(comerciantes_file.most_recent_sales_range.describe())
print(comerciantes_file.most_recent_purchases_range.describe())

count     334696
unique         2
top            N
freq      327657
Name: category_1, dtype: object
count     334696
unique         2
top            N
freq      238596
Name: category_4, dtype: object
count    334696.000000
mean          2.295474
std           1.596626
min           0.000000
25%           1.000000
50%           1.000000
75%           4.000000
max           5.000000
Name: category_2, dtype: float64
count     334696
unique         5
top            E
freq      177104
Name: most_recent_sales_range, dtype: object
count     334696
unique         5
top            E
freq      175309
Name: most_recent_purchases_range, dtype: object


In [42]:
# Transformar os dados categórios em variáveis dummies
comerciantes_file = pd.get_dummies(comerciantes_file, columns = ['category_1', 'category_2', 'category_4', 'most_recent_sales_range', 'most_recent_purchases_range']);comerciantes_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 32 columns):
merchant_id                      334696 non-null object
numerical_1                      334696 non-null float64
numerical_2                      334696 non-null float64
avg_sales_lag3                   334696 non-null float64
avg_purchases_lag3               334696 non-null float64
avg_sales_lag6                   334696 non-null float64
avg_purchases_lag6               334696 non-null float64
avg_sales_lag12                  334696 non-null float64
avg_purchases_lag12              334696 non-null float64
qtty_active_months_lag3          334696 non-null float64
qtty_active_months_lag6          334696 non-null float64
qtty_active_months_lag12         334696 non-null float64
category_1_N                     334696 non-null uint8
category_1_Y                     334696 non-null uint8
category_2_0.0                   334696 non-null uint8
category_2_1.0                   334696 n

In [43]:
# Finalmente, somente dados numéricos
comerciantes_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 32 columns):
merchant_id                      334696 non-null object
numerical_1                      334696 non-null float64
numerical_2                      334696 non-null float64
avg_sales_lag3                   334696 non-null float64
avg_purchases_lag3               334696 non-null float64
avg_sales_lag6                   334696 non-null float64
avg_purchases_lag6               334696 non-null float64
avg_sales_lag12                  334696 non-null float64
avg_purchases_lag12              334696 non-null float64
qtty_active_months_lag3          334696 non-null float64
qtty_active_months_lag6          334696 non-null float64
qtty_active_months_lag12         334696 non-null float64
category_1_N                     334696 non-null uint8
category_1_Y                     334696 non-null uint8
category_2_0.0                   334696 non-null uint8
category_2_1.0                   334696 n

In [44]:
# Analisar arquivo de histórico de cartões
historico_cartoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 14 columns):
authorized_flag         199999 non-null object
card_id                 199999 non-null object
city_id                 199999 non-null int64
category_1              199999 non-null object
installments            199999 non-null int64
category_3              199390 non-null object
merchant_category_id    199999 non-null int64
merchant_id             199075 non-null object
month_lag               199999 non-null int64
purchase_amount         199999 non-null float64
purchase_date           199999 non-null object
category_2              185232 non-null float64
state_id                199999 non-null int64
subsector_id            199999 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 21.4+ MB


In [45]:
# Com base no dicionário de dados, será adotada a seguinte estratégia para os arquivos transacoes_historicas.csv
# e novas_transacoes_comerciantes.csv

# Variáveis que penalizam um cartão: authorized_flag = N
# Variáveis que qualificam um cartão: authorized_flag = Y; installments = quanto maior melhor; 
# month_lag = quanto menor melhor
# Obter: Purchase date (data da compra): a menor de cada cartão

# Excluir demais variáveis:
historico_cartoes.drop(['city_id', 'category_1', 'merchant_category_id', 'merchant_id', 'purchase_amount', 'category_2', 'category_3', 'state_id', 'subsector_id'], axis=1, inplace = True)
historico_comerci.drop(['city_id', 'category_1', 'merchant_category_id', 'merchant_id', 'purchase_amount', 'category_2', 'category_3', 'state_id', 'subsector_id'], axis=1, inplace = True)

In [46]:
# Verificar se histórico possui valores NA
print(historico_cartoes.isna().sum())
print(historico_comerci.isna().sum())

authorized_flag    0
card_id            0
installments       0
month_lag          0
purchase_date      0
dtype: int64
authorized_flag    0
card_id            0
installments       0
month_lag          0
purchase_date      0
dtype: int64


In [47]:
historico_cartoes.head()

,authorized_flag,card_id,installments,month_lag,purchase_date
0,Y,C_ID_4e6213e9bc,0,-7,2017-07-15 12:10:45
1,Y,C_ID_4e6213e9bc,0,-6,2017-08-09 22:04:29
2,Y,C_ID_4e6213e9bc,0,-5,2017-09-02 10:06:26
3,Y,C_ID_4e6213e9bc,0,-11,2017-03-10 01:14:19
4,Y,C_ID_4e6213e9bc,0,0,2018-02-24 08:45:05


In [48]:
historico_comerci.head()

,authorized_flag,card_id,installments,month_lag,purchase_date
0,Y,C_ID_415bb3a509,1,1,2018-03-11 14:57:36
1,Y,C_ID_415bb3a509,1,1,2018-03-19 18:53:37
2,Y,C_ID_415bb3a509,1,2,2018-04-26 14:08:44
3,Y,C_ID_415bb3a509,1,1,2018-03-07 09:43:21
4,Y,C_ID_ef55cf8d4b,1,1,2018-03-22 21:07:53


In [49]:
# Alterar a coluna purchase_date para tipo datetime
historico_cartoes['new_purchase_date'] = pd.to_datetime(historico_cartoes['purchase_date'], format = "%Y-%m-%d")
historico_comerci['new_purchase_date'] = pd.to_datetime(historico_comerci['purchase_date'], format = "%Y-%m-%d")

In [50]:
# Excluir a variável purchase_date:
historico_cartoes.drop(['purchase_date'], axis=1, inplace = True)
historico_comerci.drop(['purchase_date'], axis=1, inplace = True)

In [51]:
# Verificar resultado:
print(historico_cartoes.info())
print(historico_comerci.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 5 columns):
authorized_flag      199999 non-null object
card_id              199999 non-null object
installments         199999 non-null int64
month_lag            199999 non-null int64
new_purchase_date    199999 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 7.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1963031 entries, 0 to 1963030
Data columns (total 5 columns):
authorized_flag      object
card_id              object
installments         int64
month_lag            int64
new_purchase_date    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 74.9+ MB
None


In [52]:
historico_cartoes.head()

,authorized_flag,card_id,installments,month_lag,new_purchase_date
0,Y,C_ID_4e6213e9bc,0,-7,2017-07-15 12:10:45
1,Y,C_ID_4e6213e9bc,0,-6,2017-08-09 22:04:29
2,Y,C_ID_4e6213e9bc,0,-5,2017-09-02 10:06:26
3,Y,C_ID_4e6213e9bc,0,-11,2017-03-10 01:14:19
4,Y,C_ID_4e6213e9bc,0,0,2018-02-24 08:45:05


In [53]:
historico_comerci.head()

,authorized_flag,card_id,installments,month_lag,new_purchase_date
0,Y,C_ID_415bb3a509,1,1,2018-03-11 14:57:36
1,Y,C_ID_415bb3a509,1,1,2018-03-19 18:53:37
2,Y,C_ID_415bb3a509,1,2,2018-04-26 14:08:44
3,Y,C_ID_415bb3a509,1,1,2018-03-07 09:43:21
4,Y,C_ID_ef55cf8d4b,1,1,2018-03-22 21:07:53


In [54]:
historico_cartoes.describe()

,installments,month_lag
count,199999.000000,199999.000000
mean,0.495452,-5.089705
std,0.927371,3.712166
min,-1.000000,-13.000000
25%,0.000000,-8.000000
50%,0.000000,-5.000000
75%,1.000000,-2.000000
max,12.000000,0.000000


In [55]:
historico_comerci.describe()

,installments,month_lag
count,1.963031e+06,1.963031e+06
mean,6.829643e-01,1.476515e+00
std,1.584069e+00,4.994483e-01
min,-1.000000e+00,1.000000e+00
25%,0.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00
75%,1.000000e+00,2.000000e+00
max,9.990000e+02,2.000000e+00


In [56]:
# Como os dataframes restantes possuem as mesmas variáveis, juntá-los em um novo dataframe
historico_geral = pd.concat([historico_cartoes, historico_comerci])

In [57]:
historico_geral.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2163030 entries, 0 to 1963030
Data columns (total 5 columns):
authorized_flag      object
card_id              object
installments         int64
month_lag            int64
new_purchase_date    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 99.0+ MB


In [58]:
historico_geral.isna().sum()

authorized_flag      0
card_id              0
installments         0
month_lag            0
new_purchase_date    0
dtype: int64

In [59]:
historico_geral.head()

,authorized_flag,card_id,installments,month_lag,new_purchase_date
0,Y,C_ID_4e6213e9bc,0,-7,2017-07-15 12:10:45
1,Y,C_ID_4e6213e9bc,0,-6,2017-08-09 22:04:29
2,Y,C_ID_4e6213e9bc,0,-5,2017-09-02 10:06:26
3,Y,C_ID_4e6213e9bc,0,-11,2017-03-10 01:14:19
4,Y,C_ID_4e6213e9bc,0,0,2018-02-24 08:45:05


In [60]:
# Criar a função que vai gerar as colunas da estratégia delineada
# min_installments fazer value_counts
# min_month_lag fazer absoluto na função
# max_month_lag fazer value counts
# oldest_Activation ???

def create_strategic_columns(x):
    resultado = pd.DataFrame()
    resultado['denied_purchase?'] = pd.Series([1 if 'Y' in x['authorized_flag'].unique() else 0])
    resultado['oldest_Date'] = x['new_purchase_date'].min()
    resultado['latest_Date'] = x['new_purchase_date'].max()
    resultado['min_installments'] = x['installments'].min()
    resultado['max_installments'] = x['installments'].max()
    resultado['min_month_lag'] = x['month_lag'].min()
    resultado['max_month_lag'] = x['month_lag'].max()
    return resultado

In [61]:
# Gerar dataframe com a função acima
df_cols_estrategicas = historico_geral.groupby('card_id').apply(create_strategic_columns)
df_cols_estrategicas = df_cols_estrategicas.reset_index().drop('level_1', axis=1).set_index('card_id')

In [62]:
df_cols_estrategicas.head()

,denied_purchase?,oldest_Date,latest_Date,min_installments,max_installments,min_month_lag,max_month_lag
card_id,,,,,,,
C_ID_00007093c1,1,2018-04-03 11:13:35,2018-04-09 16:23:59,1,1,2,2
C_ID_0001238066,1,2018-03-01 16:48:27,2018-04-30 19:57:30,-1,10,1,2
C_ID_0001506ef0,1,2018-03-16 22:21:58,2018-03-22 09:14:30,0,0,1,1
C_ID_0001793786,1,2017-11-15 15:44:20,2017-12-31 17:35:56,0,0,1,2
C_ID_000183fdda,1,2018-03-02 12:26:26,2018-04-30 14:59:53,-1,4,1,2


In [ ]:
# Visando unir os dataframes, alterar o índice para card_id
train_file.set_index('card_id', inplace=True)
test_file.set_index('card_id',  inplace=True)

In [ ]:
# criar dataframe de treino e teste via merge de df_cols_estrategicas e arquivos train_file e test_file
df_treino = pd.merge(train_file, df_cols_estrategicas, how='left', left_index=True, right_index=True)
df_teste  = pd.merge(test_file,  df_cols_estrategicas, how='left', left_index=True, right_index=True)

In [ ]:
# Salvar o resultado até aqui obtido em arquivo CSV:
df_treino.to_csv('df_treino_ETL.csv')
df_teste.to_csv('df_teste_ETL.csv')

In [ ]:
# Retomando o trabalho:
df_treino  = pd.read_csv('df_treino_ETL.csv')
df_teste   = pd.read_csv('df_teste_ETL.csv')

In [ ]:
df_treino.info()

In [ ]:
df_teste.info()

In [ ]:
# Recriar os campos de tipo data:
df_treino['date_Card_Activation'] = pd.to_datetime(df_treino['date_Card_Activation'], format = "%Y-%m-%d")
df_treino['oldest_Date'] = pd.to_datetime(df_treino['oldest_Date'], format = "%Y-%m-%d")
df_treino['latest_Date'] = pd.to_datetime(df_treino['latest_Date'], format = "%Y-%m-%d")
df_teste['date_Card_Activation'] = pd.to_datetime(df_teste['date_Card_Activation'], format = "%Y-%m-%d")
df_teste['oldest_Date'] = pd.to_datetime(df_teste['oldest_Date'], format = "%Y-%m-%d")
df_teste['latest_Date'] = pd.to_datetime(df_teste['latest_Date'], format = "%Y-%m-%d")

In [ ]:
# Criar colunas com resultado de cálculos com data de ativação
df_treino['oldest_Activation'] = (df_treino['oldest_Date'] - df_treino['date_Card_Activation']).dt.days
df_treino['latest_Activation'] = (df_treino['latest_Date'] - df_treino['date_Card_Activation']).dt.days

In [ ]:
df_teste['oldest_Activation'] = (df_teste['oldest_Date'] - df_teste['date_Card_Activation']).dt.days
df_teste['latest_Activation'] = (df_teste['latest_Date'] - df_teste['date_Card_Activation']).dt.days

In [ ]:
# Excluir as colunas de data:
df_treino.drop(['date_Card_Activation', 'oldest_Date', 'latest_Date'], axis=1, inplace = True)
df_teste.drop(['date_Card_Activation', 'oldest_Date', 'latest_Date'], axis=1, inplace = True)

In [ ]:
df_treino.head()

In [ ]:
df_teste.head()

In [ ]:
df_treino.info()

In [ ]:
df_teste.info()

In [ ]:
# Verificar a presença de valores NA:
print(df_treino.isna().sum())
print(df_teste.isna().sum())

In [ ]:
# características dos dados de treino:
df_treino.describe()

In [ ]:
df_teste.describe()

In [ ]:
# Correlação de Pearson
df_treino.corr(method = 'pearson')

In [ ]:
# Verificando o skew de cada atributo
df_treino.skew()

In [ ]:
# Feature Selection
# Fonte: https://www.kaggle.com/pablocastilla/predict-house-prices-with-xgboost-regression
# model = xgboost.XGBRegressor(colsample_bytree=0.4,
#                  gamma=0,                 
#                  learning_rate=0.07,
#                  max_depth=3,
#                  min_child_weight=1.5,
#                  n_estimators=10000,                                                                    
#                  reg_alpha=0.75,
#                  reg_lambda=0.45,
#                  subsample=0.6,
#                  seed=42) 

# model.fit(df_treino.iloc[:, 2:19], df_treino['target'])

# OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

# Variáveis mais importantes:
# var_mais_importantes = list(dict((k, v) for k, v in model.get_booster().get_fscore().items() if v >= 4200).keys())
# print(var_mais_importantes)
var_mais_importantes = ['latest_Activation', 'max_installments', 'oldest_Activation', 'max_month_lag', 
                        'min_installments', 'min_month_lag']